In [7]:
import pandas as pd
from os import mkdir

# Carregar o dataset
df = pd.read_csv("dados-mensais/pc-05.csv")  # Ajuste encoding e separador se necessário

# Filtrar apenas a UF 25
df_paraiba = df[df["UF"] == 25]

# Selecionar apenas as colunas desejadas
colunas_desejadas = ["UF","B0011", "B0012", "B0013", "B0014", "B0015", "B0016",
                      "B0017", "B0018", "B0019", "B00110", "B00111", "B00112",
                      "C01011", "C01012"]
df_filtrado = df_paraiba[colunas_desejadas]


display(df_filtrado)

try: 
    mkdir('resultados')
except FileExistsError:
    print('Diretório \"resultados\" já existe.')

# Exibir as primeiras linhas do dataframe filtrado
df_filtrado.to_csv("resultados/pb-05-contaminacaoXrenda.csv", index=False, encoding="latin1", sep=";")


,UF,B0011,B0012,B0013,B0014,B0015,B0016,B0017,B0018,B0019,B00110,B00111,B00112,C01011,C01012
87619,25,2,2,2,2,2,2,2,2,2,2,2,2,1.0,160.0
87620,25,2,2,2,2,2,2,2,2,2,2,2,2,NaN,NaN
87621,25,2,2,2,2,2,2,2,2,2,2,2,2,4.0,1570.0
87622,25,2,2,2,2,2,2,2,2,2,2,2,2,5.0,2090.0
87623,25,2,2,2,2,2,2,2,2,2,2,2,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96273,25,2,1,2,2,1,2,2,2,2,2,1,2,NaN,NaN
96274,25,2,2,2,2,2,2,2,2,2,2,2,2,NaN,NaN
96275,25,2,2,2,2,2,2,2,2,2,2,2,2,2.0,450.0
96276,25,2,2,2,2,2,2,2,2,2,2,2,2,4.0,1045.0


Diretório "resultados" já existe.
